## Function Calling 示例 1：加法计算器


In [24]:

import openai
from openai import OpenAI
import json
import os

# pip install python-dotenv
# from dotenv import load_dotenv, find_dotenv
# _=load_dotenv(find_dotenv())

# defaults to getting the key using os.environ.get("OPENAI_API_KEY")
# if you saved the key under a different environment variable name, you can do something like:
# client = OpenAI(
#   api_key=os.environ.get("CUSTOM_ENV_NAME"),
# )
client = OpenAI()

def get_completion(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,  # 模型输出的随机性，0 表示随机性最小
        functions=[{  # 用 JSON 描述函数。可以定义多个，但是只有一个会被调用，也可能都不会被调用
            "name": "sum",
            "description": "计算数组中所有数字的和",
            "parameters": {
                "type": "object",
                "properties": {
                    "numbers": {
                        "type": "array",
                        "items": {
                            "type": "number"
                        }
                    }
                }
            },
        }],
    )
    return response.choices[0].message

prompt = "Tell me the sum of 1, 2, 3, 4, 5, 6, 7, 8, 9, 10."
# prompt = "桌上有 2 个苹果，四个桃子和 3 本书，一共有几个水果？"
#prompt = "1+2+3+4+...+99+100=?"
messages = [
    {"role": "system", "content": "你是一个小学数学老师，你要教学生加法"},
    {"role": "user", "content": prompt}
]
response = get_completion(messages)
messages.append(response)  # 把大模型的回复加入到对话中
print(response)

# 如果返回的是函数调用结果，则打印出来
# if (response.get("function_call")):
#     # 是否要调用 sum
#     if (response["function_call"]["name"] == "sum"):
#         args = json.loads(response["function_call"]["arguments"])
#         result = sum(args["numbers"])
#         print(result)
#         messages.append(
#             {"role": "function", "name": "sum", "content": str(result)})  # 整数 result，必须转成字符串
#         print(get_completion(messages).content)


ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"numbers":[1,2,3,4,5,6,7,8,9,10]}', name='sum'), tool_calls=None)


In [40]:
# json.loads(response.function_call.arguments)
# response["function_call"]["arguments"]  # TypeError: 'ChatCompletionMessage' object is not subscriptable
# response.get("function_call")           # AttributeError: 'ChatCompletionMessage' object has no attribute 'get'
response

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "numbers": [6, 3]\n}', name='multiply'), tool_calls=None)

## Function Calling 实例 2：四则混合运算计算器


In [29]:
def get_completion(messages, model="gpt-4"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,  # 模型输出的随机性，0 表示随机性最小
        functions=[  # 用 JSON 描述函数。可以定义多个，但是只有一个会被调用，也可能都不会被调用
            {
                "name": "sum",
                "description": "计算数组中所有数字的和",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "numbers": {
                            "type": "array",
                            "items": {
                                "type": "number",
                                "description": "必须是数值类型"
                            }
                        }
                    }
                },
            },
            {
                "name": "subtract",
                "description": "计算 a - b 的值",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "a": {
                            "type": "number",
                            "description": "被减数，必须是数值类型"
                        },
                        "b": {
                            "type": "number",
                            "description": "减数，必须是数值类型"
                        }
                    }
                },
            },
            {
                "name": "multiply",
                "description": "计算数组中所有数字的积",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "numbers": {
                            "type": "array",
                            "items": {
                                "type": "number",
                                "description": "必须是数值类型"
                            }
                        }
                    }
                },
            },
            {
                "name": "divide",
                "description": "计算 a/b 的值",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "a": {
                            "type": "number",
                            "description": "被除数，必须是数值类型"
                        },
                        "b": {
                            "type": "number",
                            "description": "除数，必须是数值类型"
                        }
                    }
                },
            }
        ],
    )
    return response.choices[0].message

prompt = "6 * 3 / (4+2) = ?"
# prompt = "桌上有 2 个苹果，四个桃子和 3 本书，水果比书多多少？"
# prompt = """
# 让我们一步步计算：小明在一家水果店买水果。他买了X斤苹果，每斤10元；4斤香蕉，每斤5元；
# 和3斤橙子，每斤8元。他手头有100元。请问小明买完这些水果后，他还剩下多少钱？
#"""

messages = [
    {"role": "system", "content": "你是一个小学数学老师，你要教学生四则混合运算"},
    {"role": "user", "content": prompt}
]

response = get_completion(messages)
messages.append(response)  # 把大模型的回复加入到对话中。非常重要！
print(response)
while (response.get("function_call")):
    # 是否要调用 sum
    args = json.loads(response["function_call"]["arguments"])
    function_name = response["function_call"]["name"]

    if (function_name == "sum"):
        result = sum(args["numbers"])
    elif (function_name == "subtract"):
        result = args["a"] - args["b"]
    elif (function_name == "multiply"):
        result = 1
        for number in args["numbers"]:
            result *= number
    elif (function_name == "divide"):
        result = args["a"] / args["b"]
    else:
        result = "Unknown function"

    print(result)
    messages.append(
        {"role": "function", "name": function_name, "content": str(result)})
    response = get_completion(messages)
    messages.append(response)  # 把大模型的回复加入到对话中
    print(response)

print(response.content)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "numbers": [6, 3]\n}', name='multiply'), tool_calls=None)


AttributeError: 'ChatCompletionMessage' object has no attribute 'get'

In [39]:
# json.loads(response.tostr())
# "function_call" in response
response.get("function_call")

AttributeError: 'ChatCompletionMessage' object has no attribute 'get'

In [ ]:
## Function Calling 实例 5：地图

import requests

amap_key ="6d672e6194caa3b639fccf2caf06c342"



In [5]:
# 任务描述
instruction = """
你的任务是陪人聊天，擅长使用心理学原理及性格分析为对方解决问题。
"""

# 用户输入
input_text = """
我啥时候能结婚呢？
"""

# 这是系统预置的 prompt。魔法咒语的秘密都在这里
prompt = f"""
{instruction}

用户输入：
{input_text}
"""

response = get_completion(prompt)
print(response)

BadRequestError: Error code: 400 - {'error': {'message': "'\\n\\n你的任务是陪人聊天，擅长使用心理学原理及性格分析为对方解决问题。\\n\\n\\n用户输入：\\n\\n我啥时候能结婚呢？\\n\\n' is not of type 'array' - 'messages'", 'type': 'invalid_request_error', 'param': None, 'code': None}}